In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import fftconvolve
import IPython
import pyroomacoustics as pra
import soundfile as sf
import shutil
from pathlib import Path


In [7]:
def apply_rir(input_wav, corners, output_path):
    # Read WAV file
    fs, signal = wavfile.read(input_wav)
    
    # Create room and set parameters
    room = pra.Room.from_corners(corners, fs=fs, max_order=3, materials=pra.Material(0.2, 0.15), ray_tracing=True, air_absorption=True)
    room.extrude(2., materials=pra.Material(0.2, 0.15))
    room.set_ray_tracing(receiver_radius=0.5, n_rays=10000, energy_thres=1e-5)
    
    # Add source and microphone
    room.add_source([1.0, 2.0, 1.0], signal=signal)
    R = np.array([[2.95, 3.05], [2.0, 2.0], [0.5,  0.5]])  # [[x], [y], [z]]
    room.add_microphone(R)
    
    # Simulate room
    room.image_source_model()
    room.simulate()
    
    # Save the processed audio to the output path
    wavfile.write(output_path, fs, room.mic_array.signals[0, :])


In [8]:
def convert_flac_to_wav_and_back(input_flac, output_flac, corners):
    # Temporary file paths
    temp_wav = input_flac.replace('.flac', '.wav')
    processed_wav = temp_wav.replace('.wav', '_processed.wav')
    
    # Read FLAC file
    signal, fs = sf.read(input_flac)
    
    # Write to temporary WAV file
    wavfile.write(temp_wav, fs, signal)
    
    # Apply RIR to the WAV file
    apply_rir(temp_wav, corners, processed_wav)
    
    # Read the processed WAV file
    processed_signal, fs = sf.read(processed_wav)
    
    # Write to the output FLAC file
    sf.write(output_flac, processed_signal, fs)
    
    # Remove temporary files
    os.remove(temp_wav)
    os.remove(processed_wav)


In [9]:
def process_all_flac_files(base_dir, corners):
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.flac'):
                input_flac = os.path.join(root, file)
                relative_path = os.path.relpath(input_flac, base_dir)
                output_flac = os.path.join(base_dir, 'processed', relative_path)
                
                # Ensure output directory exists
                os.makedirs(os.path.dirname(output_flac), exist_ok=True)
                
                # Process the file
                convert_flac_to_wav_and_back(input_flac, output_flac, corners)

# Define the corners of the room
corners = np.array([[0,0], [0,4], [5,4], [5,0]]).T  # [x,y]

# Define the base directory containing the FLAC files
base_dir = '/home/kuzurman/v-cloaka/flac'

# Process all FLAC files
process_all_flac_files(base_dir, corners)
